# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846881863644                   -0.70    4.8         
  2   -7.852315333211       -2.26       -1.53    1.0   23.0ms
  3   -7.852616703560       -3.52       -2.56    1.5   25.3ms
  4   -7.852645866164       -4.54       -2.87    2.8   31.8ms
  5   -7.852646492134       -6.20       -3.17    1.0   23.3ms
  6   -7.852646679846       -6.73       -4.33    1.2   24.0ms
  7   -7.852646686669       -8.17       -4.86    2.5   32.2ms
  8   -7.852646686719      -10.30       -5.29    1.0   23.5ms
  9   -7.852646686729      -11.00       -6.05    1.2   24.2ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846791275082                   -0.70           4.5         
  2   -7.852524500502       -2.24       -1.63   0.80    2.0    237ms
  3   -7.852636586260       -3.95       -2.71   0.80    1.0   21.5ms
  4   -7.852646314646       -5.01       -3.28   0.80    2.2   28.1ms
  5   -7.852646678302       -6.44       -4.16   0.80    1.2   22.7ms
  6   -7.852646686435       -8.09       -4.76   0.80    1.5   24.0ms
  7   -7.852646686720       -9.55       -5.56   0.80    1.5   24.7ms
  8   -7.852646686730      -11.03       -6.28   0.80    2.0   26.8ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.357283e+01     3.378443e+00
 * time: 0.4149448871612549
     1     1.068290e+00     1.860881e+00
 * time: 0.6045629978179932
     2    -1.034102e+00     2.352222e+00
 * time: 0.6270649433135986
     3    -3.598764e+00     1.888202e+00
 * time: 0.6592779159545898
     4    -4.446392e+00     2.025039e+00
 * time: 0.6915860176086426
     5    -6.340751e+00     1.224626e+00
 * time: 0.7236628532409668
     6    -7.016797e+00     1.238681e+00
 * time: 0.7561540603637695
     7    -7.203477e+00     1.362639e+00
 * time: 0.7785179615020752
     8    -7.490388e+00     1.303596e+00
 * time: 0.8009140491485596
     9    -7.697591e+00     3.260601e-01
 * time: 0.8330168724060059
    10    -7.781480e+00     1.787088e-01
 * time: 0.8555960655212402
    11    -7.823763e+00     8.613847e-02
 * time: 0.8779828548431396
    12    -7.840357e+00     4.694750e-02
 * time: 0.9008569717407227
    13    -7.844084e+00     5.222326e-02
 * time: 0.9232580661

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846860838342                   -0.70    5.0         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645914412                   -1.64         
  2   -7.852646686730       -6.11       -3.71    1.57s
  3   -7.852646686730      -13.34       -7.25    176ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 6.340684110238205e-7
|ρ_newton - ρ_scfv| = 1.5603379024289852e-7
|ρ_newton - ρ_dm|   = 3.036986212984515e-9
